# Тестовое

#### - Выполненное задание должно полностью выполняться через Run All без ошибок, находять при этом в папке Тестовое. OC Windows.

Выполнил Васильев Денис

In [529]:
import pandas as pd               #библиотека для считывания файлов с данными
import numpy as np                  #библиотека для оругления значений?
import matplotlib.pyplot as plt     #библиотека для графиков
from scipy import stats as st       #библиотека для проверки гипотез
import math as mth
import seaborn as sns
from io import StringIO
#import plotly.express as px
#from plotly import graph_objects as go

## 1. Задание

Написать функцию которая удаляет дубликаты из списка сохраняя его порядок. На вход ф-ции подается список, на выходе 

In [584]:
ls = ['one', 'two', 'one', 'two', 'three', 'two', 'one', 'four', 'three', 'five']

In [585]:
def remove_duplicates(ls):
    seen = set()
    result = []
    
    for item in ls:
        if item not in seen:
            seen.add(item)
            result.append(item)
            
    return result

# Проверка функции
print(remove_duplicates(ls))

['one', 'two', 'three', 'four', 'five']


## 2. Задание

- В папке Reports находятся excel файлы с выгрузками из базы данных по каждому дню, файлы имеют схожее название типа - "DAG-111_patients_2023-02-01.xlsx" имеют общую структуру, могут иметь несколько листов данных. Их нужно объединить в один ДатаСет и в нем должна содержатся колонка, в которой информация из, какого файла загружена запись

- Файлы со значениями анализов пациентов за 2022 и 2023 год. Нужно посчитать среднее значение показателей пациентов за 2023 год общее и по каждому пациенту в отдельности.

Файл имеет 2 столбца:
    id - id пациента
    values - перечисленные анализы пациента разделенные ";"

- Помимо предложеного расчета, предложите свой\свои варианты.

In [535]:
#function функция для разделения excel файла


def myfunction(file): 

    sheets_data = pd.read_excel(file, sheet_name=None)
    file_name = file.split('/')[-1]

    dataset = pd.DataFrame(columns = ['date', 'value', 'id', 'file_name'])
    # Можно перебрать листы и вывести заголовки столбцов каждого листа
    for sheet_name, df in sheets_data.items():
        #print(f"\nЛист '{sheet_name}' имеет следующие колонки:")
        #print(df.columns.tolist())
        #print(df)
        #print(df['values'].str.split(';').explode().str.split(' - '))
        a = df['values'].str.split(';').explode().str.split(' - ')
        res = pd.DataFrame(a.to_list(), columns = ['date', 'value'])
        res['id'] = int(df['id'])
        res['file_name'] = file_name
        #dataset = dataset.append(res)
        dataset = pd.concat([dataset, res])
            
    return dataset

путь к файлам dag

In [536]:
dag1_excel = '/Users/denisvasilev/Documents/Тестовое/DAG-111_patients_2023-02-01.xlsx'

In [537]:
dag2_excel = '/Users/denisvasilev/Documents/Тестовое/DAG-111_patients_2023-02-02.xlsx'

In [538]:
dag3_excel = '/Users/denisvasilev/Documents/Тестовое/DAG-111_patients_2023-02-03.xlsx'

In [539]:
#используем функцию для вывода всех листов excel файла
dag1 = myfunction(dag1_excel)

In [540]:
#заменяем , на .
dag1['value']= dag1['value'].str.replace(',', '.')

In [541]:
#заменяем тип данных obj на float
dag1['value'] = dag1['value'].astype(float)

In [542]:
#убераем пустые символы перед и после даты
dag1['date'] = dag1['date'].str.strip()

In [543]:
dag2 = myfunction(dag2_excel)
dag2['value']= dag2['value'].str.replace(',', '.')

In [544]:
#удаляем лишнее 
dag2['value'] = dag2['value'].replace({' \(Хилезная сыворотка\)': ''}, regex = True)

In [545]:
dag2['value'] = dag2['value'].astype(float)
dag2['date'] = dag2['date'].str.strip()

In [546]:
dag2.head()

,date,value,id,file_name
0,2022-06-07,127.00,9,DAG-111_patients_2023-02-02.xlsx
1,2022-09-16,120.18,9,DAG-111_patients_2023-02-02.xlsx
2,2023-05-12,130.56,9,DAG-111_patients_2023-02-02.xlsx
0,2022-07-01,80.10,6,DAG-111_patients_2023-02-02.xlsx
1,2022-12-03,88.30,6,DAG-111_patients_2023-02-02.xlsx


In [547]:
dag3 = myfunction(dag3_excel)
dag3['value']= dag3['value'].str.replace(',', '.')

In [548]:
#удаляем лишнее 
dag3 = dag3.query ("value != 'Хил'")

In [549]:
dag3['value'] = dag3['value'].astype(float)
dag3['date'] = dag3['date'].str.strip()

In [550]:
dag3.head()

,date,value,id,file_name
0,2022-09-08,69.04,4,DAG-111_patients_2023-02-03.xlsx
1,2022-09-29,78.10,4,DAG-111_patients_2023-02-03.xlsx
3,2022-11-01,66.77,4,DAG-111_patients_2023-02-03.xlsx
4,2023-03-22,73.30,4,DAG-111_patients_2023-02-03.xlsx
0,2022-02-26,89.00,3,DAG-111_patients_2023-02-03.xlsx


соединяем три файла dag1, dag2 и dag3

In [551]:
dag123 = pd.concat([dag1, dag2, dag3])

делаем срез за 2023 год

In [552]:
dag123_2023 = dag123.query("date.str.contains('2023')")

считаем общее среднее значение по пациентам за 2023 год

In [553]:
mean_value_all = round(dag123_2023['value'].mean(),2)
print('общее среднее значение по пациентам за 2023 год =', mean_value_all)

общее среднее значение по пациентам за 2023 год = 80.99


считаем среднее значение показателей по каждому пациенту в отдельности

In [554]:
dag123_2023_user = dag123_2023.groupby('id')['value'].mean()
print('среднее значение показателей по каждому пациенту в отдельности', dag123_2023_user)

среднее значение показателей по каждому пациенту в отдельности id
1     72.500000
2     85.400000
3     90.920000
4     73.300000
5     67.000000
6     80.900000
7     58.713333
8     70.300000
9    130.560000
Name: value, dtype: float64


## 3. Задание

- В папке visits файл посещения. Столбцу "Список пройденных исследований" соответствует "Список дат пройденных исследований".
-  Необходимо произвести очистку от дублей и оставить последние актуальные анализы(каждому уникальному анализу должна соответствовать последняя дата)

In [555]:
#посещения
visits_excel = '/Users/denisvasilev/Documents/Тестовое/посещения.xlsx'

In [556]:
pos = pd.read_excel(visits_excel)

In [557]:
pos = pos[['Список пройденных исследований', 'Список дат пройденных исследований']]

In [558]:
pos.columns = ['analysis', 'dates']

In [559]:
pos

,analysis,dates
0,ESLI.LI.C.1.21 - Исследование микроальбумина в...,2023-08-03; 2023-08-04; 2023-08-03; 2023-08-01...
1,ESLI.LI.C.1.21 - Исследование микроальбумина в...,2023-08-03; 2023-08-04; 2023-08-03
2,243 - Эхокардиография; 12 - ЭКГ,2023-11-13; 2023-08-30
3,ESLI.LI.C.1.21 - Исследование микроальбумина в...,2023-10-26; 2023-10-02; 2023-10-02; 2023-10-02...
4,25 - Кардиология; ESLI.LI.C.2.2 - Общий клинич...,2023-10-21; 2023-10-04; 2023-10-03; 2023-10-03...
5,ESLI.LI.C.1.16 - Липидный профиль (комплекс); ...,2023-10-28; 2023-10-28; 2023-10-28; 2023-10-27...
6,ESLI.LI.1.19.1 - Определение гемоглобина А1с (...,2023-11-17; 2023-05-02; 2023-04-28; 2023-04-28...
7,ESLI.LI.1.8.3 - Определение креатинина; ESLI.L...,2023-11-23; 2023-11-23; 2023-11-23; 2023-11-23...


In [560]:
t1 = pos['analysis'].str.split(';').explode().str.split('-')

In [561]:
t1.count()

48

In [562]:
t2 = pos['dates'].str.split(';').explode()

In [563]:
t2.count()

46

не совпадает кол-во строк

In [564]:
#выбираем код анализа
t1 = t1.apply(lambda x: x[0] )

In [565]:
t1[4].unique()

array(['25 ', ' ESLI.LI.C.2.2 ',
       ' микроскопическое исследование мазка крови при выявлении патологии',
       ' ESLI.LI.1.47.7 ', ' ESLI.LI.1.42.13 ', ' ESLI.LI.C.1.16 ',
       ' ESLI.LI.1.8.3 ', ' ESLI.LI.C.1.1 ', ' 12 ', ' 243 '],
      dtype=object)

In [566]:
#удаляем значение, которое не является кодом анализа
t1 = t1[t1 != ' микроскопическое исследование мазка крови при выявлении патологии']

In [567]:
t1.count()

46

In [568]:
pos['dates'].str.split(';').explode().count()

46

In [569]:
table = pd.concat([t1,t2], axis = 1)

In [570]:
table[table.duplicated()]

,analysis,dates
1,ESLI.LI.C.1.21,2023-08-03
1,12,2023-08-04
1,243,2023-08-03


In [571]:
table['analysis'] = table['analysis'].str.strip()

In [572]:
# Получаем для каждого анализа последнюю дату
table.groupby('analysis')['dates'].max()

analysis
12                  2023-11-22
243                 2023-11-13
25                  2023-10-21
ESLI.LI.1.19.1      2023-11-17
ESLI.LI.1.4.5       2023-10-27
ESLI.LI.1.42.13     2023-10-03
ESLI.LI.1.47.7      2023-11-23
ESLI.LI.1.8.1       2023-04-28
ESLI.LI.1.8.3       2023-11-23
ESLI.LI.1.8.7       2023-04-28
ESLI.LI.C.1.1       2023-10-03
ESLI.LI.C.1.16      2023-10-28
ESLI.LI.C.1.2       2023-11-23
ESLI.LI.C.1.21      2023-10-26
ESLI.LI.C.2.2       2023-10-04
Name: dates, dtype: object

## 4. Задание

- В папке visits файл посещения.
- В папке visits файл словарь.

- Необходимо произвести категоризацию диагнозов из столбца "Код мкб-10" файла посещения:
  - Создать словарь из файла словарь, колонка "Группа ДН" - ключ, колонка "kod_mkb_10" - значения.
  - Вывести результат в новом столбце("dn") файла посещения - без дублей, сортированный

посещения

In [573]:
visits = pd.read_excel(visits_excel)
visits

,ИД пациента,Дата диспансерного приема,Код мкб-10,Список пройденных исследований,Список дат пройденных исследований
0,123,2023-11-09,I11.0,ESLI.LI.C.1.21 - Исследование микроальбумина в...,2023-08-03; 2023-08-04; 2023-08-03; 2023-08-01...
1,1234,2023-11-27,I11.0,ESLI.LI.C.1.21 - Исследование микроальбумина в...,2023-08-03; 2023-08-04; 2023-08-03
2,12345,2023-11-25,I11.9,243 - Эхокардиография; 12 - ЭКГ,2023-11-13; 2023-08-30
3,12,2023-11-01,I11.9,ESLI.LI.C.1.21 - Исследование микроальбумина в...,2023-10-26; 2023-10-02; 2023-10-02; 2023-10-02...
4,1,2023-11-13,I25.1;I11.9,25 - Кардиология; ESLI.LI.C.2.2 - Общий клинич...,2023-10-21; 2023-10-04; 2023-10-03; 2023-10-03...
5,5654,2023-11-29,I11.9,ESLI.LI.C.1.16 - Липидный профиль (комплекс); ...,2023-10-28; 2023-10-28; 2023-10-28; 2023-10-27...
6,9,2023-11-30,E11.9;I11.9,ESLI.LI.1.19.1 - Определение гемоглобина А1с (...,2023-11-17; 2023-05-02; 2023-04-28; 2023-04-28...
7,456,2023-11-23,I25.1,ESLI.LI.1.8.3 - Определение креатинина; ESLI.L...,2023-11-23; 2023-11-23; 2023-11-23; 2023-11-23...


словарь

In [574]:
dictionary = pd.read_excel('/Users/denisvasilev/Documents/Тестовое/словарь.xlsx')
dictionary 

,Группа ДН,kod_mkb_10
0,СД2,E11
1,СД2,E11.2
2,СД2,E11.3
3,СД2,E11.4
4,СД2,E11.5
...,...,...
137,ЯБЖ,K25.7
138,ЯБЖ,K25.9
139,ЯБДК,K26
140,ЯБДК,K26.7


In [575]:
#создание словаря
# поменял местами в задание Ключ и Значение!!!!!
# Ключ - Код МКБ-10, Значение - Группа ДН
diagnosis_dict = dictionary.set_index('Группа ДН')['kod_mkb_10'].to_dict()
diagnosis_dict

{'СД2': 'E11.9',
 'ГХС': 'E78.2',
 'АГ': 'I15.9',
 'ОИМ': 'I25.8',
 'ИБС': 'I25.9',
 'МА': 'I48.9',
 'ХСН': 'I50.9',
 'ОНМК': 'G46.8',
 'Последствия ОНМК': 'I69.8',
 'ХОБЛ': 'J44.9',
 'ЯБЖ': 'K25.9',
 'ЯБДК': 'K26.9'}

In [576]:
diagnosis_dict = dictionary.set_index('kod_mkb_10')['Группа ДН'].to_dict()
diagnosis_dict

{'E11': 'СД2',
 'E11.2': 'СД2',
 'E11.3': 'СД2',
 'E11.4': 'СД2',
 'E11.5': 'СД2',
 'E11.6': 'СД2',
 'E11.7': 'СД2',
 'E11.8': 'СД2',
 'E11.9': 'СД2',
 'E78': 'ГХС',
 'E78.0': 'ГХС',
 'E78.2': 'ГХС',
 'I10': 'АГ',
 'I11': 'АГ',
 'I11.0': 'АГ',
 'I11.9': 'АГ',
 'I12': 'АГ',
 'I12.0': 'АГ',
 'I12.9': 'АГ',
 'I13': 'АГ',
 'I13.0': 'АГ',
 'I13.1': 'АГ',
 'I13.2': 'АГ',
 'I13.9': 'АГ',
 'I15': 'АГ',
 'I15.0': 'АГ',
 'I15.1': 'АГ',
 'I15.2': 'АГ',
 'I15.8': 'АГ',
 'I15.9': 'АГ',
 'I21': 'ОИМ',
 'I21.0': 'ОИМ',
 'I21.1': 'ОИМ',
 'I21.2': 'ОИМ',
 'I21.3': 'ОИМ',
 'I21.4': 'ОИМ',
 'I21.9': 'ОИМ',
 'I22': 'ОИМ',
 'I22.0': 'ОИМ',
 'I22.1': 'ОИМ',
 'I22.8': 'ОИМ',
 'I22.9': 'ОИМ',
 'I24': 'ОИМ',
 'I24.0': 'ОИМ',
 'I24.8': 'ОИМ',
 'I24.9': 'ОИМ',
 'I20.0': 'ОИМ',
 'I25.8': 'ОИМ',
 'I20.1': 'ИБС',
 'I20.8': 'ИБС',
 'I20.9': 'ИБС',
 'I25.0': 'ИБС',
 'I25.1': 'ИБС',
 'I25.2': 'ИБС',
 'I25.3': 'ИБС',
 'I25.4': 'ИБС',
 'I25.5': 'ИБС',
 'I25.6': 'ИБС',
 'I25.9': 'ИБС',
 'I48': 'МА',
 'I48.0': 'МА',
 'I48

In [577]:
#категоризация диагнозов присвоим новой колонке dn соответствующее значение из словаря

In [578]:
# разделим значение код мкб-10 у пациентов, у которых несколько диагнозов
codes = visits['Код мкб-10'].str.split(';').explode()
codes

0    I11.0
1    I11.0
2    I11.9
3    I11.9
4    I25.1
4    I11.9
5    I11.9
6    E11.9
6    I11.9
7    I25.1
Name: Код мкб-10, dtype: object

In [579]:
#категоризируем из словаря 
codes = codes.map(diagnosis_dict)
codes

0     АГ
1     АГ
2     АГ
3     АГ
4    ИБС
4     АГ
5     АГ
6    СД2
6     АГ
7    ИБС
Name: Код мкб-10, dtype: object

In [580]:
#соединим обратно категории по индексу
codes = codes.groupby(level=0).agg(';'.join)
codes

0        АГ
1        АГ
2        АГ
3        АГ
4    ИБС;АГ
5        АГ
6    СД2;АГ
7       ИБС
Name: Код мкб-10, dtype: object

In [581]:
#переименовываем колонку на dn
codes.name = 'dn'

In [582]:
#добавим категории в таблицу посетителей
visits = pd.concat([visits, codes], axis = 1)

In [583]:
visits

,ИД пациента,Дата диспансерного приема,Код мкб-10,Список пройденных исследований,Список дат пройденных исследований,dn
0,123,2023-11-09,I11.0,ESLI.LI.C.1.21 - Исследование микроальбумина в...,2023-08-03; 2023-08-04; 2023-08-03; 2023-08-01...,АГ
1,1234,2023-11-27,I11.0,ESLI.LI.C.1.21 - Исследование микроальбумина в...,2023-08-03; 2023-08-04; 2023-08-03,АГ
2,12345,2023-11-25,I11.9,243 - Эхокардиография; 12 - ЭКГ,2023-11-13; 2023-08-30,АГ
3,12,2023-11-01,I11.9,ESLI.LI.C.1.21 - Исследование микроальбумина в...,2023-10-26; 2023-10-02; 2023-10-02; 2023-10-02...,АГ
4,1,2023-11-13,I25.1;I11.9,25 - Кардиология; ESLI.LI.C.2.2 - Общий клинич...,2023-10-21; 2023-10-04; 2023-10-03; 2023-10-03...,ИБС;АГ
5,5654,2023-11-29,I11.9,ESLI.LI.C.1.16 - Липидный профиль (комплекс); ...,2023-10-28; 2023-10-28; 2023-10-28; 2023-10-27...,АГ
6,9,2023-11-30,E11.9;I11.9,ESLI.LI.1.19.1 - Определение гемоглобина А1с (...,2023-11-17; 2023-05-02; 2023-04-28; 2023-04-28...,СД2;АГ
7,456,2023-11-23,I25.1,ESLI.LI.1.8.3 - Определение креатинина; ESLI.L...,2023-11-23; 2023-11-23; 2023-11-23; 2023-11-23...,ИБС
